# Importing packages and load CNN

In [ ]:
# Load the extension
# %load_ext autoreload
# Autoreload all modules
# %autoreload 2

In [ ]:
from funcs import *

# Telling the notebook to make plots inline.
%matplotlib inline

plt.rc('text', usetex=True)
plt.rc('font', family='serif', size=12)

## Start camera and DAC

In [ ]:
# Create an instant camera object with the camera device found first
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
print("Using Camera ", camera.GetDeviceInfo().GetModelName())

camera.Open()

# Sanity checks
print('Camera Auto Gain is :', camera.GainAuto())
if camera.GainAuto() == 'Off':
    print('Safe to go ahead with the code..')
else:
    print('***Warning: Camera Auto Gain is ON!***\nTurn it off using Pylon software.')

# Set exposure
camera.ExposureTimeAbs = Exposure

In [ ]:
# Allowed number of generations
NumGens = 40
# Number of images to be grabbed.
countOfImagesToGrab = pop_per_gen * (NumGens+2)

# Start the grabbing of c_countOfImagesToGrab images.
# The camera device is parameterized with a default configuration which
# sets up free-running continuous acquisition.
camera.StartGrabbingMax(countOfImagesToGrab)

In [ ]:
# Start DAC
start_channel = 1
bus = busworks.BusWorks_DAC()
bus.start()
print("DAC on IP: ", bus.address)

bus.read_voltages()

## Initial State

In [ ]:
Img_initial = Capture_image2(camera, 500)
print('Initial condition: {}'.format(Img_initial.sum()/n_pixl**2))
plt.imshow(Img_initial, cmap=cm.binary_r)
plt.colorbar()

In [ ]:
# Initial condition of beam
current_beam_status = np.zeros(num_params)

## Alignment loop

In [ ]:
Range = Range_orig

In [ ]:
# Creating the initial population of deltas
new_pop_deltas = sample_d(Range, shape=pop_size, first_sample=True)
print("\nRange:", Range, "x waist")
# sess = tf.Session()

gen = 0
mode_stability = 0
mode_old = (50,50)
# dont_alter_z_cm = False
t0 = time.time()

# GA loop
for gen in range(NumGens):
    # Shrink range
    Range *= shrink_factor
    print("\n\n Gen: {} Range: {} x waist".format(gen+1, Range))
    # if dont_alter_z_cm:
    #     new_pop_deltas[:,-1] = 0.
    if gen == 0:
        current_beam_status, new_pop_deltas, fitness = calc_pop_fitness(current_beam_status, \
        new_pop_deltas, fitness, camera, bus, only_offsprings=False)
    else:
        current_beam_status, new_pop_deltas, fitness = calc_pop_fitness(current_beam_status, \
        new_pop_deltas, fitness, camera, bus, only_offsprings=True)
    # Selecting the best parents in the population for mating.
    current_beam_status, parents, parents_fitness, Img_is_saturated, image = select_mating_pool(current_beam_status, new_pop_deltas, fitness, num_parents_mating, t0, gen, camera, bus, \
                           show_the_best=True, save_best=True)
    # Generating next generation using crossover.
    offspring_crossover = crossover(parents, (pop_per_gen - num_parents_mating, num_params))
    # Adding some variations to the offsrping using mutation.
    offspring_mutation = mutation(current_beam_status, offspring_crossover, Range)
    # Creating the new population based on the parents and offspring.
    new_pop_deltas[:num_parents_mating, :] = parents
    new_pop_deltas[num_parents_mating:, :] = offspring_mutation
    fitness[:num_parents_mating] = parents_fitness
    fitness[num_parents_mating:] *= 0.
    # # Scan the cavity and regain the max power point in z_CM
    # current_beam_status, new_pop_deltas, img = scan_cavity(current_beam_status, new_pop_deltas, \
    #                                                        Range, Sz, camera, bus, show_fig=True)
#     mode_new = Find_mode2(image, separation1=5, Width=30, thresh=0.35, corner=0, show_fig=True, show_basis=True)
#     print("Mode: ", mode_new)
#     if gen > 0:
#         if np.all(mode_new == mode_old):
#             mode_stability += 1
#         else:
#             mode_stability = 0
#         # check mode stability
#         if mode_stability == 3 and np.any(mode_new != (0,0)):
#             pass
#             # print('Jumping to fundamental mode..')
#             # current_beam_status, new_pop_deltas, _ = jump_2_fundamental(current_beam_status, new_pop_deltas, \
#             #                                                             mode_new, camera, bus, show_fig=True)
#         elif mode_stability > 3 and np.any(mode_new != (0,0)):
#             pass
#             # print('Trying alternate jump..')
#             # current_beam_status, new_pop_deltas, _ = jump_2_fundamental(current_beam_status, new_pop_deltas, \
#             #                                                             mode_new, camera, bus, show_fig=True)
#             # print('Scanning the cavity..')
#             # current_beam_status, new_pop_deltas, img = scan_cavity(current_beam_status, new_pop_deltas, \
#             #                                                        1.1*Range, Sz, camera, bus, show_fig=True)
#         elif mode_stability == 5 and np.all(mode_new == (0,0)):
#             print('Fundamental mode identified! Exiting..')
#             break
#     print('\n*****  Mode is {} - occurances: {}  *****\n'.format(mode_new, mode_stability+1))
#     mode_old = mode_new

In [ ]:
Exposure

### Fundamental mode position and scanning

In [ ]:
# %matplotlib notebook

# Beam_status = [5.37932319e-04, 3.33051668e-04, -1.06754584e-03, -2.70645706e-03, 1.14001752e-06]

# dirn = 1
# fig, ax1 = plt.subplots(1,1, figsize=(6,6))

# for i in range(10000):
#     Beam_status += np.array([0., 0., 0., 0., dirn*1e-8])
#     if Beam_status[-1] >= phi_CM_PZT_max or Beam_status[-1] <= 0.:
#         dirn *= -1
#     R_fn1, Img1 = Reward_fn(Beam_status, camera, bus)
#     ax1.imshow(Img1, cmap=cm.binary_r)
#     fig.canvas.draw()

# Plot status

In [ ]:
VS = [2.04685336e-04, -1.35697666e-04, 1.03526137e-03, 1.61361483e-03, 0]
Set_Voltage(VS, bus)

In [ ]:
plt.interactive('off') # if this cell is run after digital locking cell (last)
Img1 = Capture_image2(camera, 300)
print(Img1.sum()/n_pixl**2)
plt.imshow(Img1[::-1], cmap=cm.binary_r)

## Make gif of the images

In [ ]:
import imageio

files = os.listdir(ImagesFolder)
files.sort()
images = []
for filename in files:
    images.append(imageio.imread(ImagesFolder+'/'+filename))
imageio.mimsave(ImagesFolder + '/movie.gif', images, duration=0.5)

In [ ]:
%matplotlib notebook

In [ ]:
# Once GA has found the fundamental mode implement basic digital locking
show_fig = True
read_time = True

lock_record = [0.]
P_record = [0.]
i = 0
j = 0

if show_fig:
    fig, (ax1, ax2) = plt.subplots(2,1, figsize=(8,6))

while True:
    tt1 = time.time()
    Img = Capture_image(300, camera)
    Mode = Find_mode2(Img, separation1=10, Sigma1=1, Width=10, thresh=0.5, corner=0)
    P_new = 100.*Img.max()/n_pixl**2./(Mode[0]+Mode[1]+1.)
    # activate loop if power falls below thresh
    if not locking_loop_on and P_new < P_thresh*P_max:
        locking_loop_on = True
    # if Power drops then reverse the direction
    if P_new < P_old:
        j += 1
        # set delay in direction reversal
        if j == 1:
            # print('reversing..')
            direction *= -1
            j = 0
    # if Power is increasing, note for how many steps did it increase
    if P_new > P_old:
        if P_new > P_max:
            P_max = P_new
            print('P_max reset to ', P_max)
        elif P_new > P_thresh*P_max:
            locking_loop_on = False
    # Actual locking loop
    if locking_loop_on:
        # take the delta step
        current_beam_status[4] += direction*z_step
        # cumulatively subtracting each delta step from all deltas
        new_pop_deltas[:,4] -= direction*z_step
        Set_Voltage(current_beam_status, bus)
        # record the step
        lock_record.append(lock_record[-1]+direction*z_step)
        P_record.append(P_new)
    # record the value before updation
    P_old = P_new
    if read_time:
        delta_t = time.time() - tt1
        read_time = False
    if show_fig and i%10==0:
        # update image
        ax1.imshow(Img, cmap=cm.binary_r)
        N_l = len(lock_record)
        ax2.clear()
        ax2.plot(np.arange(N_l)*(delta_t), np.array(lock_record), 'r')
        ax2.set_xlabel('time (sec)')
        ax2.set_ylabel('$\Delta L$')
        fig.canvas.draw()
    # print('{0:1.2f} Pnew: {1:1.3f}'.format(time.time() - tt1, P_new))
    i += 1

# Stop Camera and DAC

In [ ]:
# Stop camera and DAC
camera.Close()
bus.stop()

In [ ]:
# from scipy.interpolate import interp1d

# ii = np.where(np.array(P_record)>200)
# ff = interp1d(np.arange(len(lock_record))[ii], np.array(lock_record)[ii], kind='cubic')
# yy = ff(np.arange(102,2246))

# plt.ion()
# fig, ax1 = plt.subplots(1,1)
# ax1.plot(np.arange(102,2246)[:500], yy[:500])
# fig.canvas.draw()

In [ ]:
# Fundamental mode found at - 
# [5.37932319e-04, 3.33051668e-04, -1.06754584e-03, -2.70645706e-03, 1.14001752e-06]
# [-4.85114666e-04, -2.79346829e-06, -1.57814286e-03, -2.89688008e-03, 3.61083367e-07]